In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 작업할 내용
- 모든 데이터를 다 import한 다음 필요한 |r| > 0.4 인 항목만 추출하여 csv 파일로 정리

In [2]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc

# 그래프 기본 테마 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 복잡한 통계 처리를 위한 라이브러리
from scipy import stats
from sklearn.preprocessing import LabelEncoder


### 1번 파일

In [3]:
df107 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/1.회원정보/201807_train_회원정보.parquet')
df108 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/1.회원정보/201808_train_회원정보.parquet')
df109 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/1.회원정보/201809_train_회원정보.parquet')
df110 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/1.회원정보/201810_train_회원정보.parquet')
df111 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/1.회원정보/201811_train_회원정보.parquet')
df112 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/1.회원정보/201812_train_회원정보.parquet')

df107.head()

,기준년월,ID,남녀구분코드,연령,Segment,회원여부_이용가능,회원여부_이용가능_CA,회원여부_이용가능_카드론,소지여부_신용,소지카드수_유효_신용,...,할인금액_제휴연회비_B0M,청구금액_기본연회비_B0M,청구금액_제휴연회비_B0M,상품관련면제카드수_B0M,임직원면제카드수_B0M,우수회원면제카드수_B0M,기타면제카드수_B0M,카드신청건수,Life_Stage,최종카드발급경과월
0,201807,TRAIN_000000,2,40대,D,1,1,0,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(2),22
1,201807,TRAIN_000001,1,30대,E,1,1,1,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(1),18
2,201807,TRAIN_000002,1,30대,C,1,1,0,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀출산기,20
3,201807,TRAIN_000003,2,40대,D,1,1,0,1,2,...,0,0,0,0개,0개,0개,0개,1,자녀성장(2),17
4,201807,TRAIN_000004,2,40대,E,1,1,1,1,1,...,0,0,0,0개,0개,0개,0개,1,자녀성장(1),15


In [4]:
# 1번 데이터 병합
df1 = pd.concat([df107, df108, df109, df110, df111, df112], ignore_index=True)
df1 = df1.sort_values(by=['ID', '기준년월']).reset_index(drop=True)
df1

,기준년월,ID,남녀구분코드,연령,Segment,회원여부_이용가능,회원여부_이용가능_CA,회원여부_이용가능_카드론,소지여부_신용,소지카드수_유효_신용,...,할인금액_제휴연회비_B0M,청구금액_기본연회비_B0M,청구금액_제휴연회비_B0M,상품관련면제카드수_B0M,임직원면제카드수_B0M,우수회원면제카드수_B0M,기타면제카드수_B0M,카드신청건수,Life_Stage,최종카드발급경과월
0,201807,TRAIN_000000,2,40대,D,1,1,0,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(2),22
1,201808,TRAIN_000000,2,40대,D,1,1,0,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(2),23
2,201809,TRAIN_000000,2,40대,D,1,1,0,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(2),24
3,201810,TRAIN_000000,2,40대,D,1,1,0,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(2),25
4,201811,TRAIN_000000,2,40대,D,1,1,0,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(2),26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201808,TRAIN_399999,2,40대,E,1,1,0,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(1),14
2399996,201809,TRAIN_399999,2,40대,E,1,1,0,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(1),15
2399997,201810,TRAIN_399999,2,40대,E,1,1,0,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(1),16
2399998,201811,TRAIN_399999,2,40대,E,1,1,0,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(1),17


#### 필요한 컬럼
이용금액_R3M_신용체크
이용금액_R3M_신용
_1순위카드이용금액
_2순위카드이용금액
	기준년월	ID Segment

In [5]:
cols_to_keep = [
    '이용금액_R3M_신용체크',
    '이용금액_R3M_신용',
    '_1순위카드이용금액',
    '기준년월',
    'ID',
    'Segment'
]

df1 = df1[cols_to_keep]


In [6]:
df1.head()

,이용금액_R3M_신용체크,이용금액_R3M_신용,_1순위카드이용금액,기준년월,ID,Segment
0,196,196,3681,201807,TRAIN_000000,D
1,-1190,-1190,2529,201808,TRAIN_000000,D
2,868,868,3948,201809,TRAIN_000000,D
3,-367,-367,3157,201810,TRAIN_000000,D
4,-478,-478,3048,201811,TRAIN_000000,D


### 2번 파일 -> 0.4 이상 없음

### 3번 파일

In [7]:
df307 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/3.승인매출정보/201807_train_승인매출정보.parquet')
df308 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/3.승인매출정보/201808_train_승인매출정보.parquet')
df309 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/3.승인매출정보/201809_train_승인매출정보.parquet')
df310 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/3.승인매출정보/201810_train_승인매출정보.parquet')
df311 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/3.승인매출정보/201811_train_승인매출정보.parquet')
df312 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/3.승인매출정보/201812_train_승인매출정보.parquet')

df307.head()

,기준년월,ID,최종이용일자_기본,최종이용일자_신판,최종이용일자_CA,최종이용일자_카드론,최종이용일자_체크,최종이용일자_일시불,최종이용일자_할부,이용건수_신용_B0M,...,승인거절건수_한도초과_B0M,승인거절건수_BL_B0M,승인거절건수_입력오류_B0M,승인거절건수_기타_B0M,승인거절건수_R3M,승인거절건수_한도초과_R3M,승인거절건수_BL_R3M,승인거절건수_입력오류_R3M,승인거절건수_기타_R3M,이용금액대
0,201807,TRAIN_000000,20180719,20180713,20180719,10101,20180203,20180709,20180713,11,...,0,0,0,0,3,3,0,0,0,01.100만원+
1,201807,TRAIN_000001,20180719,20180719,20170728,20170327,10101,20180719,20171231,13,...,0,0,0,0,3,3,0,0,0,03.30만원+
2,201807,TRAIN_000002,20180706,20180706,20180706,20151119,20141230,20180706,20180627,12,...,0,0,0,0,0,0,0,0,0,01.100만원+
3,201807,TRAIN_000003,20180721,20180715,20180721,10101,20141111,20180704,20180715,6,...,0,0,0,0,3,3,0,0,0,01.100만원+
4,201807,TRAIN_000004,20180124,20180124,10101,10101,20180512,20180124,10101,-2,...,0,0,0,0,0,0,0,0,0,09.미사용


In [8]:
# 3번 데이터 병합
df3 = pd.concat([df307, df308, df309, df310, df311, df312], ignore_index=True)
df3 = df3.sort_values(by=['ID', '기준년월']).reset_index(drop=True)
df3

,기준년월,ID,최종이용일자_기본,최종이용일자_신판,최종이용일자_CA,최종이용일자_카드론,최종이용일자_체크,최종이용일자_일시불,최종이용일자_할부,이용건수_신용_B0M,...,승인거절건수_한도초과_B0M,승인거절건수_BL_B0M,승인거절건수_입력오류_B0M,승인거절건수_기타_B0M,승인거절건수_R3M,승인거절건수_한도초과_R3M,승인거절건수_BL_R3M,승인거절건수_입력오류_R3M,승인거절건수_기타_R3M,이용금액대
0,201807,TRAIN_000000,20180719,20180713,20180719,10101,20180203,20180709,20180713,11,...,0,0,0,0,3,3,0,0,0,01.100만원+
1,201808,TRAIN_000000,20180821,20180816,20180821,10101,20180203,20180816,20180713,7,...,0,0,0,0,3,3,0,0,0,01.100만원+
2,201809,TRAIN_000000,20180920,20180906,20180920,10101,20180203,20180906,20180713,7,...,0,0,0,0,0,0,0,0,0,01.100만원+
3,201810,TRAIN_000000,20181025,20181025,20181021,10101,20180203,20181025,20180713,7,...,0,0,0,0,0,0,0,0,0,01.100만원+
4,201811,TRAIN_000000,20181126,20181126,20181120,10101,20180203,20181126,20180713,7,...,0,0,0,0,0,0,0,0,0,01.100만원+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201808,TRAIN_399999,20180831,20180831,10101,10101,10101,20180831,20180713,1,...,0,0,0,0,0,0,0,0,0,03.30만원+
2399996,201809,TRAIN_399999,20180831,20180831,10101,10101,10101,20180831,20180713,0,...,0,0,0,0,0,0,0,0,0,02.50만원+
2399997,201810,TRAIN_399999,20180831,20180831,10101,10101,10101,20180831,20180713,0,...,0,0,0,0,0,0,0,0,0,03.30만원+
2399998,201811,TRAIN_399999,20180831,20180831,10101,10101,10101,20180831,20180713,-2,...,0,0,0,0,0,0,0,0,0,04.10만원+


In [9]:
cols_to_keep = [
    '정상청구원금_B0M',
    '정상청구원금_B2M',
    '이용금액_일시불_R12M',
    '이용금액_일시불_B0M',
    '이용금액_일시불_R6M',
    '이용금액_일시불_R3M',
    '이용금액_오프라인_B0M',
    '정상입금원금_B5M',
    '정상입금원금_B0M',
    '이용금액_오프라인_R3M',
    '이용금액_오프라인_R6M',
    '정상입금원금_B2M',
    '_3순위업종_이용금액',
    'ID'
]
df3 = df3[cols_to_keep]
df3.head()


,정상청구원금_B0M,정상청구원금_B2M,이용금액_일시불_R12M,이용금액_일시불_B0M,이용금액_일시불_R6M,이용금액_일시불_R3M,이용금액_오프라인_B0M,정상입금원금_B5M,정상입금원금_B0M,이용금액_오프라인_R3M,이용금액_오프라인_R6M,정상입금원금_B2M,_3순위업종_이용금액,ID
0,14440,16524,20667,1995,8380,2877,4043,9205,6335,4131,11097,16125,672,TRAIN_000000
1,15008,15712,22754,905,7041,1250,3883,11161,7685,4852,13568,15592,734,TRAIN_000000
2,15536,14440,24100,1102,8282,4002,3926,16705,8113,11853,11681,6335,835,TRAIN_000000
3,15251,15008,23647,1037,5922,3045,3906,15950,7293,11715,15847,7685,897,TRAIN_000000
4,15028,15536,24417,1063,4454,3203,3918,16718,7164,11751,16603,8113,344,TRAIN_000000


### 4번 파일

In [10]:
df407 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/4.청구입금정보/201807_train_청구정보.parquet')
df408 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/4.청구입금정보/201808_train_청구정보.parquet')
df409 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/4.청구입금정보/201809_train_청구정보.parquet')
df410 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/4.청구입금정보/201810_train_청구정보.parquet')
df411 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/4.청구입금정보/201811_train_청구정보.parquet')
df412 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/4.청구입금정보/201812_train_청구정보.parquet')

df407.head()

,기준년월,ID,대표결제일,대표결제방법코드,대표청구지고객주소구분코드,대표청구서수령지구분코드,청구서수령방법,청구서발송여부_B0,청구서발송여부_R3M,청구서발송여부_R6M,...,할인금액_청구서_B0M,상환개월수_결제일_R6M,상환개월수_결제일_R3M,선결제건수_R6M,선결제건수_R3M,연체건수_R6M,연체건수_R3M,혜택수혜금액_R3M,포인트_마일리지_환산_B0M,혜택수혜금액
0,201807,TRAIN_000000,27,자동이체,미확인,당사페이앱+이메일,문자메세지,1,1,1,...,0,5,3,0,0,1,0,3,0,0
1,201807,TRAIN_000001,13,자동이체,주거지,우편,우편,1,1,1,...,163,6,3,0,0,0,0,0,0,0
2,201807,TRAIN_000002,1,자동이체,미확인,이메일,이메일,1,1,1,...,0,6,3,0,0,0,0,121,0,50
3,201807,TRAIN_000003,5,자동이체,주거지,우편,우편,1,1,1,...,0,5,3,2,0,0,0,3,0,2
4,201807,TRAIN_000004,13,자동이체,주거지,우편,우편,0,1,1,...,0,1,1,0,0,0,0,0,0,0


In [11]:
# 4번 데이터 병합
df4 = pd.concat([df407, df408, df409, df410, df411, df412], ignore_index=True)
df4 = df4.sort_values(by=['ID', '기준년월']).reset_index(drop=True)
df4

,기준년월,ID,대표결제일,대표결제방법코드,대표청구지고객주소구분코드,대표청구서수령지구분코드,청구서수령방법,청구서발송여부_B0,청구서발송여부_R3M,청구서발송여부_R6M,...,할인금액_청구서_B0M,상환개월수_결제일_R6M,상환개월수_결제일_R3M,선결제건수_R6M,선결제건수_R3M,연체건수_R6M,연체건수_R3M,혜택수혜금액_R3M,포인트_마일리지_환산_B0M,혜택수혜금액
0,201807,TRAIN_000000,27,자동이체,미확인,당사페이앱+이메일,문자메세지,1,1,1,...,0,5,3,0,0,1,0,3,0,0
1,201808,TRAIN_000000,27,자동이체,미확인,당사페이앱+이메일,문자메세지,1,1,1,...,0,5,3,0,0,1,0,3,0,0
2,201809,TRAIN_000000,27,자동이체,미확인,당사페이앱+이메일,문자메세지,1,1,1,...,0,6,3,0,0,0,0,0,0,0
3,201810,TRAIN_000000,27,자동이체,미확인,당사페이앱+이메일,문자메세지,1,1,1,...,0,6,3,0,0,0,0,0,0,0
4,201811,TRAIN_000000,27,자동이체,미확인,당사페이앱+이메일,문자메세지,1,1,1,...,0,6,3,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201808,TRAIN_399999,23,자동이체,주거지,우편,우편,1,1,1,...,0,6,3,0,0,0,0,0,0,0
2399996,201809,TRAIN_399999,23,자동이체,주거지,우편,우편,1,1,1,...,0,6,3,0,0,0,0,0,0,0
2399997,201810,TRAIN_399999,23,자동이체,주거지,우편,우편,1,1,1,...,0,6,3,0,0,0,0,0,0,0
2399998,201811,TRAIN_399999,23,자동이체,주거지,우편,우편,1,1,1,...,0,6,3,0,0,0,0,0,0,0


In [12]:
cols_to_keep = [
    '청구금액_R6M',
    '청구금액_R3M',
    '청구금액_B0',
    'ID'
]

df4 = df4[cols_to_keep]
df4.head()


,청구금액_R6M,청구금액_R3M,청구금액_B0,ID
0,88693,46588,12226,TRAIN_000000
1,119419,53336,15756,TRAIN_000000
2,132269,43715,15732,TRAIN_000000
3,92304,45716,14227,TRAIN_000000
4,97254,43918,13958,TRAIN_000000


### 5:8번 파일에는 |r|>0.5 없음

### outer join

In [13]:
# optimize_dataframe_memory 함수 정의
def optimize_dataframe_memory(df):
    for col in df.columns:
        col_type = df[col].dtype
        if 'int' in str(col_type):
            if df[col].isnull().any():
                 df[col] = pd.to_numeric(df[col], downcast='float')
            else:
                 df[col] = df[col].astype('int32')
        elif 'float' in str(col_type):
            df[col] = pd.to_numeric(df[col], downcast='float')
        elif col_type == object:
            num_unique_values = len(df[col].unique())
            num_total_values = len(df[col])
            if (num_unique_values / num_total_values) < 0.5:
                df[col] = df[col].astype('category')
    return df

# 각 데이터프레임에 메모리 최적화 함수 적용
# (df1, df3, df4는 이미 로드되어 있고 불필요한 컬럼이 제거된 상태라고 가정)
df1 = optimize_dataframe_memory(df1)
df3 = optimize_dataframe_memory(df3)
df4 = optimize_dataframe_memory(df4)
# df5는 이번 병합 과정에서 제외되므로 optimize_dataframe_memory 함수를 호출할 필요가 없습니다.

# 첫 번째 merge: df1과 df3 병합
df_merged = df1.merge(df3, on='ID', how='left')
# df1과 df3는 이제 더 이상 필요 없으므로 메모리에서 해제
del df1
del df3

# 두 번째 merge: df_merged와 df4 병합
# 이제 이 결과가 최종 병합된 데이터프레임이 됩니다.
df_final = df_merged.merge(df4, on='ID', how='left') # 이름을 df_final로 바로 변경
# df_merged와 df4는 이제 더 이상 필요 없으므로 메모리에서 해제
del df_merged
del df4

# 최종 df_final 데이터프레임 결과 확인
print(df_final.head())


/tmp/ipython-input-13-614347863.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('int32')
/tmp/ipython-input-13-614347863.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('int32')
/tmp/ipython-input-13-614347863.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

   이용금액_R3M_신용체크  이용금액_R3M_신용  _1순위카드이용금액    기준년월            ID Segment  \
0            196          196        3681  201807  TRAIN_000000       D   
1            196          196        3681  201807  TRAIN_000000       D   
2            196          196        3681  201807  TRAIN_000000       D   
3            196          196        3681  201807  TRAIN_000000       D   
4            196          196        3681  201807  TRAIN_000000       D   

   정상청구원금_B0M  정상청구원금_B2M  이용금액_일시불_R12M  이용금액_일시불_B0M  ...  이용금액_오프라인_B0M  \
0       14440       16524          20667          1995  ...           4043   
1       14440       16524          20667          1995  ...           4043   
2       14440       16524          20667          1995  ...           4043   
3       14440       16524          20667          1995  ...           4043   
4       14440       16524          20667          1995  ...           4043   

   정상입금원금_B5M  정상입금원금_B0M  이용금액_오프라인_R3M  이용금액_오프라인_R6M  정상입금원금_B2M  \
0        

In [15]:
df_final.to_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/인공지능 모델 구축/r05.parquet', index=False)
print("df_final 데이터프레임이 'r05.parquet' 파일로 성공적으로 저장되었습니다.")

df_final 데이터프레임이 'r05.parquet' 파일로 성공적으로 저장되었습니다.
